In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-dataset/Mental Health Dataset.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC, SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier

from category_encoders import LeaveOneOutEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

df=pd.read_csv("/kaggle/input/mental-health-dataset/Mental Health Dataset.csv")
df

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,7/27/2015 23:25,Male,United States,Business,Yes,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,Maybe,Not sure
292360,8/17/2015 9:38,Male,South Africa,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes
292361,8/25/2015 19:59,Male,United States,Business,No,Yes,No,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,No
292362,9/26/2015 1:07,Male,United States,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes


In [4]:
df.iloc[0,:]

Timestamp                  8/27/2014 11:29
Gender                              Female
Country                      United States
Occupation                       Corporate
self_employed                          NaN
family_history                          No
treatment                              Yes
Days_Indoors                     1-14 days
Growing_Stress                         Yes
Changes_Habits                          No
Mental_Health_History                  Yes
Mood_Swings                         Medium
Coping_Struggles                        No
Work_Interest                           No
Social_Weakness                        Yes
mental_health_interview                 No
care_options                      Not sure
Name: 0, dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Timestamp                292364 non-null  object
 1   Gender                   292364 non-null  object
 2   Country                  292364 non-null  object
 3   Occupation               292364 non-null  object
 4   self_employed            287162 non-null  object
 5   family_history           292364 non-null  object
 6   treatment                292364 non-null  object
 7   Days_Indoors             292364 non-null  object
 8   Growing_Stress           292364 non-null  object
 9   Changes_Habits           292364 non-null  object
 10  Mental_Health_History    292364 non-null  object
 11  Mood_Swings              292364 non-null  object
 12  Coping_Struggles         292364 non-null  object
 13  Work_Interest            292364 non-null  object
 14  Social_Weakness     

In [6]:
df.duplicated().sum()

2313

In [7]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [8]:
df.isnull().sum()

Timestamp                     0
Gender                        0
Country                       0
Occupation                    0
self_employed              5193
family_history                0
treatment                     0
Days_Indoors                  0
Growing_Stress                0
Changes_Habits                0
Mental_Health_History         0
Mood_Swings                   0
Coping_Struggles              0
Work_Interest                 0
Social_Weakness               0
mental_health_interview       0
care_options                  0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
# df.drop("Timestamp",axis=1)
df.shape
df.isnull().sum()

Timestamp                  0
Gender                     0
Country                    0
Occupation                 0
self_employed              0
family_history             0
treatment                  0
Days_Indoors               0
Growing_Stress             0
Changes_Habits             0
Mental_Health_History      0
Mood_Swings                0
Coping_Struggles           0
Work_Interest              0
Social_Weakness            0
mental_health_interview    0
care_options               0
dtype: int64

In [11]:
X=df.drop("treatment",axis=1)
y=df["treatment"]

X.shape,y.shape
X

y
X.head(1)

,Timestamp,Gender,Country,Occupation,self_employed,family_history,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes


In [12]:

le = LabelEncoder()

df["Gender"]=le.fit_transform(df["Gender"])


df["Country"]=le.fit_transform(df["Country"])
df["Country"]


3         34
4         34
5         25
6          0
7         34
          ..
292359    34
292360    29
292361    34
292362    34
292363    34
Name: Country, Length: 284858, dtype: int64

In [13]:
le=LabelEncoder()

column_object=df.select_dtypes("object").columns
column_object

for col in column_object:
    df[col]=le.fit_transform(df[col])

df

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
3,41,0,34,1,0,1,1,0,2,1,2,2,0,1,2,0,2
4,46,0,34,1,0,1,1,0,2,1,2,2,0,1,2,1,2
5,50,0,25,1,0,0,1,0,2,1,2,2,0,1,2,0,1
6,52,0,0,1,0,1,1,0,2,1,2,2,0,1,2,1,1
7,53,0,34,1,0,0,0,0,2,1,2,2,0,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,36,1,34,0,1,1,1,1,1,0,1,1,1,1,0,0,1
292360,37,1,29,0,0,1,1,1,1,0,1,1,1,1,0,1,2
292361,38,1,34,0,0,1,0,1,1,0,1,1,1,1,0,1,0
292362,564,1,34,0,0,1,1,1,1,0,1,1,1,1,0,1,2


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 284858 entries, 3 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   Timestamp                284858 non-null  int64
 1   Gender                   284858 non-null  int64
 2   Country                  284858 non-null  int64
 3   Occupation               284858 non-null  int64
 4   self_employed            284858 non-null  int64
 5   family_history           284858 non-null  int64
 6   treatment                284858 non-null  int64
 7   Days_Indoors             284858 non-null  int64
 8   Growing_Stress           284858 non-null  int64
 9   Changes_Habits           284858 non-null  int64
 10  Mental_Health_History    284858 non-null  int64
 11  Mood_Swings              284858 non-null  int64
 12  Coping_Struggles         284858 non-null  int64
 13  Work_Interest            284858 non-null  int64
 14  Social_Weakness          284858 non-null 

In [15]:
X=df.drop("treatment",axis=1)
X
y=df["treatment"]
X,y
X

y

3         1
4         1
5         1
6         1
7         0
         ..
292359    1
292360    1
292361    0
292362    1
292363    1
Name: treatment, Length: 284858, dtype: int64

In [16]:
# X=pd.get_dummies(data=df,columns=["Gender","Country","Occupation","self_employed" 	,"family_history" 	,"Days_Indoors","Growing_Stress", 	"Changes_Habits", 	"Mental_Health_History", 	"Mood_Swings", 	"Coping_Struggles"	,"Work_Interest","Social_Weakness","mental_health_interview" ,"care_options"])

In [17]:
X.shape

(284858, 16)

In [18]:
X.isnull().sum()

Timestamp                  0
Gender                     0
Country                    0
Occupation                 0
self_employed              0
family_history             0
Days_Indoors               0
Growing_Stress             0
Changes_Habits             0
Mental_Health_History      0
Mood_Swings                0
Coping_Struggles           0
Work_Interest              0
Social_Weakness            0
mental_health_interview    0
care_options               0
dtype: int64